In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,U.S. House expected to advance Biden's $1.9 tr...,https://www.reuters.com/article/us-health-coro...,"Fri, 05 Feb 2021 16:40:27 GMT",2021-02-05 16:40:27+00:00
9,"1,210 new COVID-19 cases, 40 new deaths report...",https://www.wlox.com/2021/02/05/new-covid-case...,"Fri, 05 Feb 2021 16:08:30 GMT",2021-02-05 16:08:30+00:00
1,What Does a Multi-Dose Series Mean for the COV...,https://www.kff.org/coronavirus-covid-19/issue...,"Fri, 05 Feb 2021 14:32:51 GMT",2021-02-05 14:32:51+00:00
2,UK backs CureVac effort to make vaccine for CO...,https://abcnews.go.com/Health/wireStory/uk-bac...,"Fri, 05 Feb 2021 14:15:00 GMT",2021-02-05 14:15:00+00:00
8,Cardiologist explains risks of heart complicat...,https://www.11alive.com/article/news/health/co...,"Fri, 05 Feb 2021 14:11:00 GMT",2021-02-05 14:11:00+00:00
7,Latest on COVID-19 in MN: Trends steady; vacci...,https://www.mprnews.org/story/2021/02/05/lates...,"Fri, 05 Feb 2021 11:30:00 GMT",2021-02-05 11:30:00+00:00
3,Where Are COVID-19 Vaccine Sites? Many Sites I...,https://www.npr.org/2021/02/05/962946721/acros...,"Fri, 05 Feb 2021 10:00:00 GMT",2021-02-05 10:00:00+00:00
5,NHL revises COVID-19 policy on arena access af...,https://www.espn.com/nhl/story/_/id/30835578/n...,"Fri, 05 Feb 2021 01:23:38 GMT",2021-02-05 01:23:38+00:00
6,Coronavirus (COVID-19) Update: FDA Announces A...,https://www.fda.gov/news-events/press-announce...,"Fri, 05 Feb 2021 00:35:04 GMT",2021-02-05 00:35:04+00:00
4,Coronavirus (COVID-19) Update: FDA Continues I...,https://www.fda.gov/news-events/press-announce...,"Thu, 04 Feb 2021 22:30:00 GMT",2021-02-04 22:30:00+00:00
